In [ ]:
from IPython.display import HTML
import ipywidgets
import itertools
import math
from matplotlib import pyplot, animation
from mpl_toolkits.mplot3d import Axes3D
import numpy

import petibmpy

%matplotlib inline

In [ ]:
pyplot.rc('font', family='serif', size=12)

In [ ]:
simudir = '.'

In [ ]:
# Parameters.
c = 1.0  # chord length
AR = 1.27  # aspect ratio (AR = 1.27 --> S / c = 1)
S = math.pi * c * AR / 4  # span
A_plan = math.pi * c * S / 4  # planform area of the plate

A_phi = numpy.radians(45.0)  # rolling amplitude
print('Rolling amplitude: A_phi = {}'.format(A_phi))

A_theta = numpy.radians(45.0)  # pitching amplitude
# A_theta = 0.0
print('Pitching amplitude: A_theta = {}'.format(A_theta))

psi = numpy.radians(90.0)  # phase difference
print('Phase difference: psi = {}'.format(psi))

theta_bias = numpy.radians(0.0)  # static pitching bias
print('Pitching bias: theta_bias = {}'.format(theta_bias))

U_inf = 1.0  # freestream velocity
R_avg = S / 2  # average rotational radius
St = 0.6  # Strouhal number
f = St * U_inf / (2 * A_phi * R_avg)  # flapping frequency
print('Flapping frequency: f = {}'.format(f))
T = 1 / f  # time period
print('Time period: T = {}'.format(T))

Re = 200.0  # Reynolds number
nu = U_inf * c / Re  # kinematic viscosity
print('Kinematic viscosity: nu = {}'.format(nu))

In [ ]:
hook = [0.0, 0.0, 0.0]  # fixed center of rotation

timestep = 250
name = 'velocity'

filepath = simudir + '/solution/{:0>7}.3D'.format(timestep)
x, y, z = petibmpy.read_body(filepath)

filepath = simudir + '/solution/{:0>7}.h5'.format(timestep)
u = petibmpy.read_field_hdf5(filepath, name)
ux, uy, uz = u[0::3], u[1::3], u[2::3]

In [ ]:
fig = pyplot.figure(figsize=(6.0, 6.0))
ax = fig.gca(projection='3d')
ax.set_xlabel('x')
ax.set_ylabel('z')
ax.set_zlabel('y')

title = ax.set_title('time step: {}'.format(timestep))
ax.scatter(*hook, depthshade=False, c='C3', marker='o', s=80)
sc = ax.scatter(x, z, y, depthshade=False, c='C1', marker='.', s=10)

start, end, s = 0, -1, 4
q = ax.quiver(x[start:end:s], z[start:end:s], y[start:end:s],
              ux[start:end:s], uz[start:end:s], uy[start:end:s],
              normalize=False, length=0.2)

ax.set_xlim(-c, c)
ax.set_ylim(2 * c, 0.0)
ax.set_zlim(-c, c)
ax.view_init(elev=0.0, azim=-90.0)
# Draw cube.
d1, d2, d3 = c, S, 2 * S * math.cos(A_phi)
r1 = [-d1 / 2, d1 / 2]
r2 = [0.0, d2]
r3 = [-d3 / 2, d3 / 2]
points = numpy.array(list(itertools.product(r1, r2, r3)))
for s, e in itertools.combinations(points, 2):
    v = numpy.sum(numpy.abs(s - e))
    if v == d1 or v == d2 or v == d3:
        ax.plot3D(*zip(s, e), color='black', linestyle=':')

fig.tight_layout();

In [ ]:
def update_figure(n, title, sc, q, display_fig=False):
    name = 'velocity'

    filepath = simudir + '/solution/{:0>7}.3D'.format(n)
    x, y, z = petibmpy.read_body(filepath)

    filepath = simudir + '/solution/{:0>7}.h5'.format(n)
    u = petibmpy.read_field_hdf5(filepath, name)
    ux, uy, uz = u[0::3], u[1::3], u[2::3]
    
    sc._offsets3d = (x, z, y)
    del(ax.collections[-1])
    start, end, s = 0, -1, 4
    q = ax.quiver(x[start:end:s], z[start:end:s], y[start:end:s],
                  ux[start:end:s], uz[start:end:s], uy[start:end:s],
                  normalize=False, length=0.2)
    title.set_text('time step: {}'.format(n))
    
    if display_fig:
        display(fig)

In [ ]:
n_slider = ipywidgets.IntSlider(value=125, min=125, max=5000, step=125,
                                description='Time step')
w = ipywidgets.interactive(update_figure, n=n_slider,
                           display_fig=ipywidgets.fixed(True),
                           title=ipywidgets.fixed(title),
                           sc=ipywidgets.fixed(sc),
                           q=ipywidgets.fixed(q))
display(w)